In [10]:
import torch
import math

def convert_to_cartesian(distances, ra, dec):
    x = distances * torch.cos(dec) * torch.cos(ra)
    y = distances * torch.cos(dec) * torch.sin(ra)
    z = distances * torch.sin(dec)
    return torch.stack((x, y, z), dim=1)

def calculate_distances(coords):
    dist_matrix = torch.cdist(coords, coords)
    unique_distances = torch.unique(dist_matrix)
    return unique_distances[unique_distances > 0]

def tetrahedron_score(coords):
    distances = calculate_distances(coords)
    if len(distances) != 2:
        return 0

    a, b = distances
    if a > b:
        a, b = b, a

    ratio = b / a
    ideal_ratio = math.sqrt(2)
    
    score = 1 - abs(ratio.item() - ideal_ratio) / ideal_ratio
    return score

def tetrahedron_score2(coords):
    distances = calculate_distances(coords)
    print(distances)
    mean_distance = torch.mean(distances)
    std_dev = torch.std(distances)

    # Assuming std_dev is small enough, the score will be close to 1
    # Otherwise, it will be closer to 0
    score = math.exp(-std_dev.item() / mean_distance.item())
    return score

# Define your input data
distances = torch.tensor([10, 15, 20, 25], dtype=torch.float32)
ra = torch.tensor([0.5, 1.0, 1.5, 2.0], dtype=torch.float32)
dec = torch.tensor([0.3, 0.6, 0.9, 1.2], dtype=torch.float32)

coords = convert_to_cartesian(distances, ra, dec)
score = tetrahedron_score(coords)

print("Tetrahedron score:", score)

Tetrahedron score: 0


In [15]:
def score_both(distances, ra, dec):
    coords = convert_to_cartesian(distances, ra, dec)
    score = tetrahedron_score(coords)
    score2 = tetrahedron_score2(coords) 
    print("Tetrahedron score:", score, score2)


distances = torch.tensor([0, 1, 0.5, 0.5])
ra = torch.tensor([0, 0, math.sqrt(3)/2, math.sqrt(3)/6])
dec = torch.tensor([0, 0, 0, math.sqrt(2/3)])
score_both(distances,ra,dec)

x_coords = torch.rand(4)
y_coords = torch.rand(4)
z_coords = torch.rand(4)
score_both(x_coords,y_coords, z_coords)

tensor([0.4616, 0.5000, 0.7704, 0.7760, 1.0000])
Tetrahedron score: 0 0.7285219772200613
tensor([0.1620, 0.3530, 0.4988, 0.5084, 0.7508, 0.9073])
Tetrahedron score: 0 0.6030645964526847


In [18]:
# chatpgpt another try
import torch

def pairwise_distances(x):
    """Compute pairwise distances between points in x."""
    diff = x.unsqueeze(2) - x.t().unsqueeze(0)
    return torch.sqrt((diff ** 2).sum(dim=1))

def tetrahedron_likeness(vertices):
    """Compute the likeness of given vertices to a perfect tetrahedron."""
    # Compute pairwise distances
    distances = pairwise_distances(vertices)
    
    # Extract unique distances (since tensor is symmetric)
    unique_distances = distances[torch.triu(torch.ones_like(distances), diagonal=1) == 1]
    
    # Sort distances
    sorted_distances = torch.sort(unique_distances)[0]
    
    # Three smallest should be the base edges and three largest the heights
    base_edges = sorted_distances[:3]
    heights = sorted_distances[3:]
    
    # Expected distances: Assume smallest distance as base edge length
    edge_length = base_edges.mean()
    expected_height = torch.sqrt(torch.tensor(2.0)/3) * edge_length
    
    # Mean squared error (MSE) between observed and expected distances
    mse_base = torch.mean((base_edges - edge_length) ** 2)
    mse_height = torch.mean((heights - expected_height) ** 2)
    
    total_mse = mse_base + mse_height
    return total_mse

# Example
vertices = torch.tensor([
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.5, torch.sqrt(torch.tensor(3.0))/2, 0.0],
    [0.5, torch.sqrt(torch.tensor(3.0))/6, torch.sqrt(torch.tensor(2.0/3))]
])

likeness_score = tetrahedron_likeness(vertices)
print(likeness_score)  # Closer to 0 is more like a perfect tetrahedron


tensor(0.0337)


In [ ]:
distances = torch.tensor([0, 1, 0.5, 0.5])
ra = torch.tensor([0, 0, math.sqrt(3)/2, math.sqrt(3)/6])
dec = torch.tensor([0, 0, 0, math.sqrt(2/3)])
vertices = torch.tensor([
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.5, torch.sqrt(torch.tensor(3.0))/2, 0.0],
    [0.5, torch.sqrt(torch.tensor(3.0))/6, torch.sqrt(torch.tensor(2.0/3))]
])

score_both(distances,ra,dec)

x_coords = torch.rand(4)
y_coords = torch.rand(4)
z_coords = torch.rand(4)
score_both(x_coords,y_coords, z_coords)